Date : 15/12/2023

## ANN for Sentiment Analysis

<img src="" alt="">

In [1]:
### --------------------
### Importing Librarires
### --------------------

import string
import re


from os import listdir


from nltk.corpus import stopwords
import nltk


from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score



import numpy as np

2023-12-22 08:08:39.572659: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-22 08:08:39.575924: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-22 08:08:39.626468: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-22 08:08:39.626506: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-22 08:08:39.626528: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 104]
[nltk_data]     Connection reset by peer>


False

### Defining the function to read files one by one

In [ ]:
# load doc into memory

def load_doc(filename):
    
    # open the file as read only
    file = open(filename, 'r')
    
    #read all text
    text = file.read()
    
    # close the file
    file.close()
    
    return text

In [6]:
# tetsting the 'load_doc' function

# reading a random file from the directory

text = load_doc('/home/dai/7. NLP & CV/NLP/review_polarity/review_polarity/txt_sentoken/pos/cv026_29325.txt')
# /home/dai/7. NLP & CV/NLP/review_polarity/review_polarity/txt_sentoken/pos/cv026_29325.txt
text


'for those of us who weren\'t yet born when the 1960\'s rock \'n\' rolled around , monterey pop affords an affectionate glimpse of the music that influenced our parents to be hippies . \nfrom otis redding to jimi hendrix , janis joplin to the mamas and the papas , and jefferson airplane to the who , this documentary is jam-packed with contagious energy . \nbut i give fair warning that i will reveal the ending , which does not do the rest of the film the justice it deserves . \nshot in 1969 at an outdoor concert that precluded woodstock , the film defies the stereotype of the general population at the time . \nsure , some have painted their faces and smoke joints , but d . a . \npennebaker ( the war room , moon over broadway ) surprisingly chooses to show a broad spectrum of the audience . \nno matter who is watching , it all comes back to the talented musicians that stir your soul . \nthe excitement starts before the music even begins . \na young girl is cleaning thousands of seats and

In [7]:
#Turn a doc into clean token
def clean_doc(doc):
    #Split ino tokens by white space
    tokens=doc.split()
    #Prepare regax for char filtering
    re_punc=re.compile('[%s]' % re.escape(string.punctuation))
    #remove punctuation from each words
    tokens=[re_punc.sub('',w)for w in tokens]
    #remove remaining tokens that are not alphabetic
    tokens=[word for word in tokens if word.isalpha()]
    #filter out stop words
    stop_words=set(stopwords.words('english'))
    tokens=[w for w in tokens if not w in stop_words]
    #Filter out short tokens
    tokens=[word for word in tokens if len(word)>1]
    return tokens

In [8]:
clean=clean_doc(text)
len(clean)

261

In [9]:
# load doc, clean and return line of tokens
def doc_to_line(filename,vocab):
    #load the doc
    doc=load_doc(filename)
    #clean doc
    tokens=clean_doc(doc)
    #Filter the vocab
    tokens=[w for w in tokens if w in vocab]
    return ' '.join(tokens)

In [11]:
vocab=open('/home/dai/7. NLP & CV/NLP/vocab.txt')
vocab=vocab.read().split()

In [12]:
doc_to_line('review_polarity/txt_sentoken/pos/cv026_29325.txt',vocab)

FileNotFoundError: [Errno 2] No such file or directory: 'review_polarity/txt_sentoken/pos/cv026_29325.txt'

In [13]:
#Seprating traing and test data
def process_train(directory,vocab):
    documents=list()
     # to check files in directory
    for filename in listdir(directory):
        if not filename.startswith('cv9'):
            path=directory + '/'+filename
            doc=load_doc(path)
            tokens=clean_doc(doc)
            documents.append(tokens)
    return documents
    
    
# Test data
def process_test(directory,vocab):
    documents=list()
     # to check files in directory
    for filename in listdir(directory):
        if filename.startswith('cv9'):
            path=directory + '/'+filename
            doc=load_doc(path)
            tokens=clean_doc(doc)
            documents.append(tokens)
    return documents

### Function to read all the docs

In [14]:
def process_docs(directory,vocab,is_train):
    documents = list()
    #walks through all the files in the folder
    for filename in listdir(directory):
        #skip any reviews in test set
        if is_train and filename.startswith('cv9'):
            continue
        if not is_train and not filename.startswith('cv9'):
            continue
        #create a full path of the file to open
        path = directory + '/'+filename
        #load the doc
        doc = load_doc(path)
        #clean the data
        tokens = clean_doc(doc)
        #load the data
        documents.append(tokens)
    return documents

In [16]:
lines = process_docs('/home/dai/7. NLP & CV/NLP/review_polarity/review_polarity/txt_sentoken/pos', vocab, False)

len(lines)

100

In [19]:
# load and clean a dataset


def load_clean_dataset(vocab, is_train):
    
    # load documents
    neg = process_docs('/home/dai/7. NLP & CV/NLP/review_polarity/review_polarity/txt_sentoken/neg',
                      vocab, is_train)
    
    pos = process_docs('/home/dai/7. NLP & CV/NLP/review_polarity/review_polarity/txt_sentoken/pos',
                      vocab, is_train)
    
    docs = neg + pos
    
    # prepare labels
    
    labels = [0 for _ in range(len(neg))] + [1 for _ in range(len(pos))]
    
    return docs, labels

In [21]:
train, train_labels = load_clean_dataset(vocab, True)
test, test_labels = load_clean_dataset(vocab, False)

In [22]:
len(train), len(train_labels)

(1800, 1800)

In [23]:
len(test), len(test_labels)

(200, 200)

## Build ANN model

In [24]:
# define the model



def define_model(n_words):
    
    
    
    # define network
    
    model = Sequential()
    
    model.add(Dense(50,
                   input_shape = (n_words,),
                   activation = 'relu'
                   )
             )
    
    model.add(Dense(1,
                   activation = 'sigmoid'
                   )
             )
    
    
    
    # compile network
    
    model.compile(loss = 'binary_crossentropy',
                 optimizer = 'adam',
                 metrics = ['accuracy']
                 )
    
    
    
    # summarize the defined model
    
    model.summary()
    
    
    
    
    # Plotting the model
    
    plot_model(model, 
              to_file = 'model.png',
              show_shapes = True
              )

    
    
    return model

In [25]:
# model call

define_model(100)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                5050      
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 5101 (19.93 KB)
Trainable params: 5101 (19.93 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
# fit a tokenizer

def create_tokenizer(lines):
    
    tokenizer = Tokenizer()
    
    tokenizer.fit_on_texts(lines)
    
    return tokenizer



# Create the tokenizer

tokenizer = create_tokenizer(train)

In [ ]:
# encode data

x_train = tokenizer.texts_to_matrix(train,
                                   mode = 'binary')

x_train.shape

: 

In [ ]:
x_test = tokenizer.texts_to_matrix(test,
                                   mode = 'binary')

x_test.shape

: 

In [ ]:
tokenizer.word_docs

: 

In [ ]:
#define network

n_words=x_train.shape[1]

model=define_model(n_words)

: 

In [ ]:
model.fit(x_train,np.array(train_labels),batch_size=10,epochs=10)

: 

In [ ]:
# Plotting the architecture of the model

plot_model(model, 
           show_dtype = True,
          show_layer_activations = True,
          show_shapes = True,
          show_layer_names = True)

: 

In [ ]:
model.evaluate(x_test, np.array(test_labels), batch_size = 1)

: 

In [ ]:
text11 = 'Best movie ever! It was great, I will definitely recommend it.'
text22 = 'This is a bad movie'

: 

def predict_sentiment(review):
    
    #clean
    token = clean_doc(review)
    
    #convert to line
    line = ' '.join(token)
    
    #encode
    encoded = tokenizer.texts_to_matrix((line), mode = 'binary')
    
    #predict sentiment
    yhat = model.predict(encoded, verbose = 0)
    
    # retrieve predicted percentage and label
    percent_pos = yhat[0,0]
    
    if round(percent_pos) == 0:
        
        return (1 - percent_pos), 'NEGATIVE'
    
    return percent_pos, 'POSITIVE'
    

In [ ]:
def predict_data(text,vocab):
    
    tokens = clean_doc(text)
    tokens = [w for w in tokens if w in vocab]
    tokens = ' '.join(tokens)
    x_new = tokenizer.texts_to_matrix([tokens],mode = 'binary')
    y_pred = model.predict(x_new,verbose=0)
    if round(y_pred[0][0])==1:
        print("Positive Review: ", y_pred[0][0])
    elif(round(y_pred[0][0])==0):
        print("Negative Review: ", y_pred[0][0])
    

: 

In [ ]:
predict_data(text22,vocab)
print(percentage,sentiment)

: 

In [ ]:
predict_data(text11, vocab)

: 

: 